In [ ]:
from notebook_utils import *

b_ts, t_ts = 0, 0

# Temporal

In [ ]:
cam_T_P = [[-4,0],[-3,0],[-2,0],[-1,0],[0,0]]
bev_T_P = [[-4,0],[-3,0],[-2,0],[-1,0],[0,0]]
Tout = len(bev_T_P)
with initialize(version_base="1.3", config_path="../configs/"):
    cfg = compose(
        config_name="train.yaml",
        return_hydra_config=True,
        overrides=[
            f"data.cam_T_P={cam_T_P}",
            f"data.bev_T_P={bev_T_P}",
            "data.version=mini",
            "data.keep_input_binimg=True",
            "data.keep_input_centr_offs=True",
            "data.keep_input_detection=True",
            "data.keep_input_persp=True",
            "data.keep_input_sampling=True",
            "data.coeffs.bev_aug.trans_rot=[50,50,0,50,0,0]",
            "data.img_params.zoom_lim=[0.5,1.5]",
            "data.img_params.rot_lim=[-45,45]",
            "data.img_params.min_visibility=2",
        ],
    )

    cfg.paths.root_dir = str(
        pyrootutils.find_root(search_from="../", indicator=".project-root")
    )

In [ ]:
datamodule = hydra.utils.instantiate(cfg.data)
datamodule.setup()
dataloader = datamodule.train_dataloader()
iter_dataloader = iter(dataloader)


In [ ]:
data = next(iter_dataloader)
data = datamodule.on_after_batch_transfer(data, 0)

In [ ]:
keys = list(data.keys())
keys.sort()
keys

In [ ]:
# data["egoTin_to_seq"][b_ts]

In [ ]:
# data["bev_aug"][b_ts]

In [ ]:
# data["egoTout_to_seq"][b_ts]

In [ ]:
px.imshow(
    data["offsets"][b_ts, :, 0],
    facet_col=0,
    facet_col_wrap=Tout,
    binary_string=True,
    width=600,
    height=300,
)

In [ ]:
px.imshow(
    data["binimg"][b_ts, :, 0],
    facet_col=0,
    facet_col_wrap=Tout,
    binary_string=True,
    width=600,
    height=300,
)

In [ ]:
# Imgs
grid = make_grid(
    data["imgs"][b_ts].flatten(0, 1),
    normalize=True,
    pad_value=1,
    padding=3,
    nrow=6,
)
show(grid)


In [ ]:
from pointbev.utils.imgs import prepare_to_render_bbox_egopose

px.imshow(
    prepare_to_render_bbox_egopose(data["bbox_egopose"][b_ts], True)
    .transpose(0, 1)
    .flip(0, 1),
    binary_string=True,
)


# Dataset

In [ ]:
with initialize(version_base="1.3", config_path="../configs/"):
    cfg = compose(
        config_name="train.yaml",
        return_hydra_config=True,
        overrides=[
            "data.version=mini",
            "data.keep_input_binimg=True",
            "data.keep_input_centr_offs=True",
            "data.keep_input_detection=True",
            "data.keep_input_persp=True",
            "data.keep_input_sampling=True",
            "data.keep_input_offsets_map=True",
            "data.keep_input_lidar=True",
            "data.kernel_scales=[1,2]",
            "data.train_shuffle=True",
            "data.coeffs.bev_aug.trans_rot=[20,20,0,20,0,0]",
            "data.img_params.zoom_lim=[0.5,1.5]",
            "data.img_params.rot_lim=[-45,45]",
            "data.img_params.min_visibility=1",
        ],
    )

    cfg.paths.root_dir = str(
        pyrootutils.find_root(search_from="../", indicator=".project-root")
    )


In [ ]:
datamodule = hydra.utils.instantiate(cfg.data)
datamodule.setup()
datamodule.visualise_mode = True
dataloader = datamodule.train_dataloader()
iter_dataloader = iter(dataloader)

In [ ]:
data = next(iter_dataloader)
data = datamodule.on_after_batch_transfer(data, 0)

## Cameras

In [ ]:
# Imgs
grid = make_grid(data["imgs"][b_ts, t_ts], normalize=True, scale_each=True)
show(grid)

In [ ]:
# Imgs-Mask
grid_mask = make_grid(
    data["persp_imgs"][b_ts, t_ts].unsqueeze(1).float(), normalize=True, scale_each=True
)
show(grid_mask)

In [ ]:
# Image-Mask
fig = go.Figure()
alphabg = 0.3
alphafg = 0.2
blendarray = (
    alphabg * grid.permute(1, 2, 0)
    + alphafg * (1 - alphabg) * grid_mask.permute(1, 2, 0)
) / (alphabg + alphafg * (1 - alphabg))
px.imshow(blendarray)

## BEV

In [ ]:
# Binimg and valid_binimg
px.imshow(
    torch.stack(
        [
            data["binimg"][b_ts, t_ts].squeeze(0),
            data["binimg_aug"][b_ts, t_ts].squeeze(0),
            data["lidar_img"][b_ts, t_ts].squeeze(0),
            data["lidar_img_aug"][b_ts, t_ts].squeeze(0),
            data["valid_binimg"][b_ts, t_ts].squeeze(0),
            data["valid_binimg_aug"][b_ts, t_ts].squeeze(0),
            ((1 - data["binimg_aug"]) * data["valid_binimg_aug"])[b_ts, t_ts].squeeze(
                0
            ),
        ],
        dim=0,
    ),
    facet_col=0,
    facet_col_wrap=7,
    binary_string=True,
    width=1500,
)

In [ ]:
# Mobility
mobility = data["mobility"][b_ts, t_ts].float()
mobility_aug = data["mobility_aug"][b_ts, t_ts].float()
px.imshow(
    torch.stack(
        [
            (mobility == 0),
            (mobility == 1),
            (mobility == 2),
            (mobility == 3),
            (mobility_aug == 0),
            (mobility_aug == 1),
            (mobility_aug == 2),
            (mobility_aug == 3),
        ],
        dim=1,
    ).squeeze(0),
    facet_col=0,
    facet_col_wrap=4,
)

In [ ]:
# Offsets
data["offsets"][data["offsets"]==255] = 0
data["offsets_aug"][data["offsets_aug"]==255] = 0
px.imshow(
    torch.cat([data["offsets"][b_ts, t_ts], data["offsets_aug"][b_ts, t_ts]]),
    facet_col=0,
    facet_col_wrap=2,
    width=600,
)


In [ ]:
# Centerness
px.imshow(
    torch.cat(
        [
            data["centerness"][b_ts, t_ts],
            data["valid_centerness"][b_ts, t_ts],
            (data["valid_centerness"] * data["centerness"])[b_ts, t_ts],
            data["centerness_aug"][b_ts, t_ts],
            data["valid_centerness_aug"][b_ts, t_ts],
            (data["valid_centerness_aug"] * data["centerness_aug"])[b_ts, t_ts],
        ]
    ).squeeze(0),
    facet_col=0,
    facet_col_wrap=3,
    width=700,
)


In [ ]:
# Offsets map
px.imshow(
    torch.cat(
        [data["offsets_map"][b_ts, t_ts], data["offsets_map_aug"][b_ts, t_ts]]
    ),
    facet_col=0,
    facet_col_wrap=2,
    width=600,
)


In [ ]:
# Offsets map distance
px.imshow(
    torch.cat(
        [
            (data["offsets_map_dist"][b_ts, t_ts]).float(),
            data["offsets_map_dist_aug"][b_ts, t_ts],
        ]
    ),
    facet_col=0,
    facet_col_wrap=2,
    width=600,
)

In [ ]:
# HDMap
if "hdmap" in data.keys():
    px.imshow(
        data["hdmap"][b_ts, t_ts].permute(1, 2, 0)[..., 0]
        + data["centerness"][b_ts, t_ts, 0].round() * 2
    )
